<a href="https://colab.research.google.com/github/june1st/GoogleColab/blob/master/LipNet_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LipNetのテスト

<table class="tfo-notebook-buttons" align="left" >
 <td>
<img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
  <td>
<img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
  </td>
</table>


## 0.最初にやること  
・LipNetのスクリプトはGPUとPython2.7を使います。  
・GoogleColabratory、左上の  「編集→ノートブックの設定」で、  
　　→ ランタイムのタイプ「Python2」  
　　→ ハードウェアアクセラレータ「GPU」  
　に変更してください。

## 1. 最初に現在のGoogleColabランタイムの状況を確認

In [0]:
!ls -la
%pwd

In [0]:
!python -c 'import chainer; chainer.print_runtime_info()'

## 2. LipNetのスクリプトをダウンロード

In [0]:
#https://www.tensorflow.org/install/install_sources#common_installation_problems
!git clone https://github.com/rizkiarm/LipNet
%cd LipNet

In [0]:
!sed -i s/'dlib==19.4.0'/'dlib==19.9.0'/ setup.py
!sed -i s/'tensorflow-gpu==1.0.1'/'tensorflow-gpu==1.4.0'/ setup.py
!sed -i s/'numpy==1.12.1'/'numpy==1.14.3'/ -i setup.py
!pip install distro
!apt update
!apt install -y cmake
!apt install ffmpeg
!pip install dlib
!pip install boost
!pip install -e .

CUDA9.0のランタイム準備（LipNetは8.0を使うのでここは不要）

In [0]:
#!wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
#!dpkg -i cuda-repo-ubuntu1604_9.0.176-1_amd64.deb
#!apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
#!apt-get update
#!sudo apt-get install cuda-9-0

CUDA8.0のランタイム準備（CoocleColaboraroryはCUDA9.2を使用しているのでこれを入れる）

In [0]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
!apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-8-0

cuDNN6.0の準備（GoogleColabratoryはcuDNN7.2を使用しているのでこれを入れる）

In [0]:
!wget https://developer.download.nvidia.com/compute/machine-learning/cudnn/secure/v6/prod/8.0_20170427/Ubuntu16_04-x64/libcudnn6_6.0.21-1%2Bcuda8.0_amd64.deb?47ppyHzjzJsX25j3SY6RXR0XT6Rnoh0gTcKxwLSliQEJjQFNmq_B-mVP_LlT5SWjOInhQuGNW3g27OKzdwmm2YiJeHBece-0on5WUvuPEHgSMS7bxrk2450CSuE0FQz44rgmEem7OQ7fUJOccZu9Xdhp0W3CsAC5hTgq660cvExKlYH2TBFrM2KokeW5YBB81LpGIecrYSuCO0B3LkfVNyYbHlBVdDhllnPJPnumEq8 -O libcudnn6_6.0.21-1+cuda8.0_amd64.deb
!dpkg -i libcudnn6_6.0.21-1+cuda8.0_amd64.deb

## LipReadingの実行

ダウンロードしらファイルの確認

In [0]:
!ls

モデルファイルの確認

In [0]:
!ls ./evaluation/models

使用できるサンプル動画の確認

In [0]:
!ls ./evaluation/samples/GRID

LipReadingの実行  
・./evaluation/samples/GRID/***　の部分を他のmpgに帰ると、他のサンプル動画に変えられる  
・最後に出てくる[DECODED]が結果  
　　→ bbaf2n.mpg の結果は 「bin blue at f two now」  
　

In [0]:
!./predict ./evaluation/models/overlapped-weights368.h5 ./evaluation/samples/GRID/bbaf2n.mpg

## LipReadingの結果に関して
・LipNetの出力は「command(4) + color(4) +preposition(4) + letter(25) + digit(10) + adverb(4)」の順番に限定されている  
　　　→ 認識できるものは以下のモノ　  
　　　　　command　：{bin, lay, place, set}  
　　　　　color　　　 ：{blue, green, red, white}  
　　　　　preposition  ：{at, by, in, with}  
　　　　　letter　　　 ：{A, . . . , Z}\{W}  
　　　　　digit　　　  ：{zero, . . . , nine}  
　　　　　adverb　　 ：{again, now, please, soon}  
 ・詳細は論文「https://arxiv.org/abs/1611.01599」を参照してください。